# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합-모듈화**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * Python 코드 모듈화
        * 각 모듈 코드 및 모델, 데이터파일을 일관성 있게 정리
        * .py 파일 생성 ==> 라이브러리 로딩, 각 task를 위한 함수 생성


## **1.환경설정**

* 경로 설정

구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/project6_2/'

## 2.모듈 구성하기

In [ ]:
%%writefile /content/drive/MyDrive/project6/emergency.py

import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 0. load key file------------------

#OpenAI API Key 환경 변수 설정
def load_key_file(filepath):
  with open(filepath, 'r') as file:
    return file.readline().strip()

  # API 키 로드 및 환경변수 설정
path = '/content/drive/MyDrive/project6_2/'
openai.api_key = load_key_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key


# 1-1 audio2text--------------------
def audio2text(audio_path, filename):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    file_path = os.path.join(audio_path, filename)
    try:
        with open(file_path, "rb") as audio_file:
            input_text = client.audio.transcriptions.create(
                file=audio_file,
                model="whisper-1",
                language="ko",
                response_format="text",
            )
        return input_text
    except Exception as e:
        return f"Error occurred: {str(e)}"

# 1-2 text2summary------------------

def text2summary(input_text, filename, x, y):
      # OpenAI 클라이언트 생성
      client = OpenAI()

      # 시스템 역할과 응답 형식 지정
      system_role = '''당신은 119 전화 내용을 요약하는 어시스턴트입니다.
      응답은 다음의 형식을 지켜주세요
      {"summary": \"텍스트 요약\"}
      '''



      # 입력데이터를 GPT-3.5-turbo에 전달하고 답변 받아오기
      response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
              {
                  "role": "system",
                  "content": system_role
              },
              {
                  "role": "user",
                  "content": input_text
              }
          ]
      )

      # 응답 받기
      answer = response.choices[0].message.content

      # 응답형식을 정리하고 return
      summary = eval(answer)["summary"]

      # 결과 데이터프레임 생성
      result = {
          "Filename": filename,
          "Summary": summary,
          "위도": x,
          "경도": y
        }

      return result


# 2. model prediction------------------

def predict(text, model, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 입력 문장 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 각 텐서를 GPU로 이동

    # 모델 예측
    with torch.no_grad():
        outputs = model(**inputs)

    # 로짓을 소프트맥스로 변환하여 확률 계s산
    logits = outputs.logits
    probabilities = logits.softmax(dim=1)

    # 가장 높은 확률을 가진 클래스 선택
    pred = torch.argmax(probabilities, dim=-1).item()
    predicted_grade = pred + 1

    return predicted_grade


def add_grade(result ,path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    save_directory = path + "fine_tuned_bert"

    # 모델 로드
    loaded_model = AutoModelForSequenceClassification.from_pretrained(save_directory).to(device)

    # 토크나이저 로드
    loaded_tokenizer = AutoTokenizer.from_pretrained(save_directory)

    predicted_grade = predict(result['Summary'], loaded_model, loaded_tokenizer)

    result["grade"] = predicted_grade

    return result


# 3-1. get_distance------------------
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers=headers, params=params)
    response = response.json()
    dist = response['route']['trafast'][0]['summary']['distance']  # m(미터)
    duration = response['route']['trafast'][0]['summary']['duration'] # 도착시간(ms)
    return dist, duration


def convert_milliseconds(df): # 변환 함수
    duration = df['duration'].copy()
    duration = duration.to_list()
    dur = []
    for ms in duration:
      ms %= 3600000
      minutes = ms // 60000
      ms %= 60000
      seconds = ms // 1000
      dur.append(f" {minutes}분 {seconds}초")
    return dur

# 3-2. recommendation------------------

def recommendation(em_list, x, y, c_id, c_key):
    dist = []
    duration = []
    distrange = 0.1
    neighbor = em_list.loc[
    (abs(em_list['위도'] - x) <= distrange) &
    (abs(em_list['경도'] - y) <= distrange)].copy()

    if(len(neighbor)<=3):
      neighbor = em_list.loc[
      (abs(em_list['위도'] - x) <= distrange + 0.1) &
      (abs(em_list['경도'] - y) <= distrange + 0.1)].copy()

    for lat, long in zip(neighbor['위도'], neighbor['경도']):
        di, du = get_dist(x, y, lat, long, c_id, c_key)
        dist.append(di)
        duration.append(du)

    neighbor['dist'] = dist
    neighbor['duration'] = duration
    return neighbor.sort_values(by='dist').head(3), neighbor.sort_values(by='duration').head(3)



Writing /content/drive/MyDrive/project6/emergency.py
